In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!sudo apt-get update -qq 2>&1 > /dev/null
!sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
!google-drive-ocamlfuse

In [ ]:
!sudo apt-get install -qq w3m # to act as web browser 
!xdg-settings set default-web-browser w3m.desktop # to set default browser
%cd /content
!mkdir drive
%cd drive
!mkdir MyDrive
%cd ..
%cd ..
!google-drive-ocamlfuse /content/drive/MyDrive

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications import ResNet50

HEIGHT = 512
WIDTH = 512

base_model = ResNet50(weights='imagenet', 
                      include_top=False, 
                      input_shape=(HEIGHT, WIDTH, 3))

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

TRAIN_DIR = "/content/drive/MyDrive/Dima/2_classification/brain_data_1/Training" 
VAL_DIR = "/content/drive/MyDrive/Dima/2_classification/brain_data_1/Testing" 
BATCH_SIZE = 8

train_datagen =  ImageDataGenerator(preprocessing_function=preprocess_input, rotation_range=90, horizontal_flip=True,vertical_flip=True)
#train_datagen =  ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    color_mode="rgb", 
    target_size=(HEIGHT, WIDTH), 
    batch_size=BATCH_SIZE) # set as training data

validation_generator = train_datagen.flow_from_directory(
    VAL_DIR, 
    color_mode="rgb", 
    target_size=(HEIGHT, WIDTH), 
    batch_size=BATCH_SIZE) # set as validation data

#train_generator = train_datagen.flow_from_directory(TRAIN_DIR, 
#                                                    target_size=(HEIGHT, WIDTH), 
#                                                    batch_size=BATCH_SIZE)

In [ ]:
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, AveragePooling2D
from tensorflow.keras.models import Sequential, Model

def build_finetune_model(base_model, dropout, fc_layers, num_classes):
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = AveragePooling2D(pool_size = (4, 4), data_format = 'channels_last')(x)
    x = Flatten()(x)
    for fc in fc_layers:
        # New FC layer, random init
        
        x = Dense(fc, activation='relu')(x) 
        x = Dropout(dropout)(x)

    # New softmax layer
    #x = Dense(1024, activation='relu')(x) 
    #x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x) 

    #x = Dense(64, activation = "relu")(x)
    #x = Dropout(0.5)(x)
    #x = Flatten(name = "flatten1")(x)
    #predictions = Dense(3, activation = "softmax")(x)
    
    finetune_model = Model(inputs=base_model.input, outputs=predictions)

    return finetune_model

#class_list = ["corona", "normal", "other"]
class_list = ["glioma", "meningioma", "notumor", "pituitary"]
FC_LAYERS = [1024, 1024]
dropout = 0.5

finetune_model = build_finetune_model(base_model, dropout=dropout, fc_layers=FC_LAYERS, num_classes=len(class_list))

In [ ]:
# Plot the training and validation loss + accuracy
def plot_training_acc(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    #loss = history.history['loss']
    #val_loss = history.history['val_loss']
    epochs = range(len(acc))

    plt.plot(epochs, acc, label='Training Accuracy')
    plt.plot(epochs, val_acc, label='Validation Accuracy')
    plt.title('Training and validation accuracy')
    plt.legend()

    # plt.figure()
    #plt.plot(epochs, loss, 'r.')
    # plt.plot(epochs, val_loss, 'r-')
    # plt.title('Training and validation loss')
    plt.savefig(fname = "/content/drive/MyDrive/Dima/2_classification/plots_data_1/img_cls_accuracy_ResNet50_8_0_0001.png", dpi = 900)
    plt.show()

    #plt.savefig('acc_vs_epochs.png')

# Plot the training and validation loss + accuracy
def plot_training_loss(history):
    #acc = history.history['accuracy']
    #val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(loss))

    #plt.plot(epochs, acc)
    #plt.plot(epochs, val_acc, 'r')
    #plt.title('Training and validation accuracy')

    # plt.figure()
    plt.plot(epochs, loss, label='Training Loss')
    plt.plot(epochs, val_loss, label='Validation Loss')
    plt.title('Training and validation loss')
    plt.legend()
    plt.savefig(fname = "/content/drive/MyDrive/Dima/2_classification/plots_data_1/img_cls_loss_ResNet50_8_0_0001.png", dpi = 900)
    plt.show()

    #plt.savefig('acc_vs_epochs.png')


In [ ]:
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt

NUM_EPOCHS = 50
BATCH_SIZE = 8
num_train_images = 8440

adam = Adam(lr=0.001)
#finetune_model.load_weights("/content/drive/MyDrive/Dima/2_classification/models_data_1/ResNet50_img_cls_weights_8_0_0001.h5")
finetune_model.compile(adam, loss='categorical_crossentropy', metrics=['accuracy'])

filepath="/content/drive/MyDrive/Dima/2_classification/models_data_1/ResNet50_img_cls_weights_8_0_0001.h5"
#checkpoint = ModelCheckpoint(filepath, monitor=["acc"], verbose=1, mode='max')
checkpoint = ModelCheckpoint(filepath, monitor="val_loss", mode="min", save_best_only = True, verbose=1)
earlystop = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 9, verbose = 1, restore_best_weights = True)
callbacks_list = [checkpoint, earlystop] 

history = finetune_model.fit(train_generator, epochs=NUM_EPOCHS, workers=8,
                             steps_per_epoch=num_train_images // BATCH_SIZE, 
                             validation_data = validation_generator,
                             shuffle=True, callbacks=callbacks_list)

In [ ]:
plot_training_acc(history)
plot_training_loss(history)

In [ ]:
history.history